In [55]:
import pandas as pd
import fitz

from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords

from bs4 import BeautifulSoup
import unicodedata
import re

import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\salsubhi1\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\salsubhi1\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\salsubhi1\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\salsubhi1\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\salsubhi1\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!


True

In [52]:
doc = fitz.open('Merged5Fashion5Flood.pdf')
text = ""
for page in doc: 
    text+= page.get_text()
print(text)

Full Terms & Conditions of access and use can be found at
https://www.tandfonline.com/action/journalInformation?journalCode=tfdt20
International Journal of Fashion Design, Technology and
Education
ISSN: (Print) (Online) Journal homepage: www.tandfonline.com/journals/tfdt20
A comparative review of zero-waste fashion design
thinking and operational research on cutting and
packing optimisation
Nesma ElShishtawy, Pammi Sinha & Julia A. Bennell
To cite this article: Nesma ElShishtawy, Pammi Sinha & Julia A. Bennell (2022) A comparative
review of zero-waste fashion design thinking and operational research on cutting and packing
optimisation, International Journal of Fashion Design, Technology and Education, 15:2,
187-199, DOI: 10.1080/17543266.2021.1990416
To link to this article:  https://doi.org/10.1080/17543266.2021.1990416
© 2022 The Author(s). Published by Informa
UK Limited, trading as Taylor & Francis
Group
Published online: 15 Nov 2021.
Submit your article to this journal 
Article vi

In [56]:
sentences = sent_tokenize(text)
print(sentences)

['Full Terms & Conditions of access and use can be found at\nhttps://www.tandfonline.com/action/journalInformation?journalCode=tfdt20\nInternational Journal of Fashion Design, Technology and\nEducation\nISSN: (Print) (Online) Journal homepage: www.tandfonline.com/journals/tfdt20\nA comparative review of zero-waste fashion design\nthinking and operational research on cutting and\npacking optimisation\nNesma ElShishtawy, Pammi Sinha & Julia A. Bennell\nTo cite this article: Nesma ElShishtawy, Pammi Sinha & Julia A. Bennell (2022) A comparative\nreview of zero-waste fashion design thinking and operational research on cutting and packing\noptimisation, International Journal of Fashion Design, Technology and Education, 15:2,\n187-199, DOI: 10.1080/17543266.2021.1990416\nTo link to this article:  https://doi.org/10.1080/17543266.2021.1990416\n© 2022 The Author(s).', 'Published by Informa\nUK Limited, trading as Taylor & Francis\nGroup\nPublished online: 15 Nov 2021.', 'Submit your article to

In [57]:
print('Number of sentences found: ' + str(len(sentences)))

Number of sentences found: 4750


In [58]:
sentences_df = pd.DataFrame(sentences)
print(sentences_df)
sentences_df.to_csv('sentences_df.csv', sep = ',')

                                                      0
0     Full Terms & Conditions of access and use can ...
1     Published by Informa\nUK Limited, trading as T...
2     Submit your article to this journal \nArticle ...
3     Both research domains seek to\nminimise waste ...
4     The\nC&P research investigates the use of math...
...                                                 ...
4745                                        Earth Syst.
4746                                               Sci.
4747                                 16 (7), 1915–1946.
4748                               3262\nM. Nied et al.
4749        / Journal of Hydrology 519 (2014) 3249–3262

[4750 rows x 1 columns]


In [59]:
sentences_df.columns = ["Sentences"]

In [60]:
sentences_df.head()

,Sentences
0,Full Terms & Conditions of access and use can ...
1,"Published by Informa\nUK Limited, trading as T..."
2,Submit your article to this journal \nArticle ...
3,Both research domains seek to\nminimise waste ...
4,The\nC&P research investigates the use of math...


In [10]:
from transformers import BertForSequenceClassification, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('./my_bert_model/')

# Load the model from `safetensors` format
model = BertForSequenceClassification.from_pretrained(
    './my_bert_model/',
    use_safetensors=True  # Ensures the model loads from `safetensors` format
)

# Save it in the `pytorch_model.bin` format
tokenizer.save_pretrained('./my_bert_model2/')
model.save_pretrained('./my_bert_model2/', safe_serialization=False)

In [61]:
from transformers import BertForSequenceClassification, AutoTokenizer
import torch
import torch.nn.functional as F  # Importing functional module for softmax

# Load the model and tokenizer
model = BertForSequenceClassification.from_pretrained('./my_bert_model2/')
tokenizer = AutoTokenizer.from_pretrained('./my_bert_model2/')

# Set the model to evaluation mode
model.eval()

# Input text for classification
input_text = pd.read_csv("sentences_df.csv") # Should be class 1 (adverse event/disaster)


# Ensure all entries in "Chunk_Text" are strings; fill NaN values if necessary
input_text["Sentences"] = input_text["Sentences"].fillna("").astype(str)


# Store the results 
predicted_classes = []
confidences = []

for sent in input_text["Sentences"]:
    # Tokenize the input text
    inputs = tokenizer(sent, return_tensors="pt", truncation=True, padding=True)

    # Run the model to get the logits (classification scores)
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits

    # Get the predicted class (e.g., 0 or 1 for binary classification)
    predicted_class = torch.argmax(logits, dim=1).item()

    # Calculate probabilities using softmax
    probabilities = F.softmax(logits, dim=1)

    # Get the confidence for the predicted class
    confidence = probabilities[0][predicted_class].item()

    # Store the results 
    predicted_classes.append(predicted_class)
    confidences.append(confidence)
    
# Add the predictions and confidence scores as new columns in the DataFrame
input_text["Predicted_Class"] = predicted_classes
input_text["Confidence"] = confidences

# Save the updated DataFrame to a new Excel file
input_text.to_excel("predicted_sentences_with_confidence.xlsx", index=False)

In [62]:
import pandas as pd

#Load the CSV file 
df = pd.read_excel("predicted_sentences_with_confidence.xlsx")

#Filter rows with Predicted_Class = 1
filtered_df = df[df["Predicted_Class"] == 1][["Sentences","Predicted_Class", "Confidence"]]

filtered_df.to_excel("filtered_output_with_Predicted_Class1.xlsx", index=False)

In [63]:
import spacy
import pandas as pd

nlp = spacy.load("en_core_web_sm")
doc = pd.read_excel("filtered_output_with_Predicted_Class1.xlsx")

doc.head(5)

,Sentences,Predicted_Class,Confidence
0,Advanced Healthcare\n−\n3DP is being used in t...,1,0.985076
1,"Due to the characteristics of the poly-\nmers,...",1,0.635592
2,Insight from research among fabric and garment...,1,0.860318
3,———Types of Floods and Floodplains——— \n2-1 \n...,1,0.999125
4,Most communities experience only a few of them.,1,0.995478


In [64]:
doc = doc.drop(['Predicted_Class','Confidence'], axis=1)
doc.head()

,Sentences
0,Advanced Healthcare\n−\n3DP is being used in t...
1,"Due to the characteristics of the poly-\nmers,..."
2,Insight from research among fabric and garment...
3,———Types of Floods and Floodplains——— \n2-1 \n...
4,Most communities experience only a few of them.


In [65]:
# Create lists to store extracted information
sentences = []
chunk_texts = []
root_texts = []
root_deps = []
root_heads = []

# Process each sentence in the "Sentences" column
for sentence in doc['Sentences']: 
    pro_doc = nlp(sentence)
    for chunk in pro_doc.noun_chunks:
        sentences.append(sentence)
        chunk_texts.append(chunk.text)
        root_texts.append(chunk.root.text)
        root_deps.append(chunk.root.dep_)
        root_heads.append(chunk.root.head.text)
        
chunk_df = pd.DataFrame({
    "Sentences": sentences,
    "Chunk_Texts":chunk_texts,
    "Root_Texts":root_texts,
    "Root_Deps":root_deps,
    "Root_Heads":root_heads
})


# Save the results to a new Excel file
chunk_df.to_excel("filtered_output_processed_noun_chunks.xlsx", index=False)

In [66]:
from transformers import BertForSequenceClassification, AutoTokenizer
import torch
import torch.nn.functional as F  # Importing functional module for softmax

# Load the model and tokenizer
model = BertForSequenceClassification.from_pretrained('./my_bert_model2/')
tokenizer = AutoTokenizer.from_pretrained('./my_bert_model2/')

# Set the model to evaluation mode
model.eval()

# Input text for classification
input_text = pd.read_excel("filtered_output_processed_noun_chunks.xlsx") # Should be class 1 (adverse event/disaster)


# Ensure all entries in "Chunk_Text" are strings; fill NaN values if necessary
input_text["Chunk_Texts"] = input_text["Chunk_Texts"].fillna("").astype(str)


# Store the results 
predicted_classes = []
confidences = []

for chunk in input_text["Chunk_Texts"]:
    # Tokenize the input text
    inputs = tokenizer(chunk, return_tensors="pt", truncation=True, padding=True)

    # Run the model to get the logits (classification scores)
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits

    # Get the predicted class (e.g., 0 or 1 for binary classification)
    predicted_class = torch.argmax(logits, dim=1).item()

    # Calculate probabilities using softmax
    probabilities = F.softmax(logits, dim=1)

    # Get the confidence for the predicted class
    confidence = probabilities[0][predicted_class].item()

    # Store the results 
    predicted_classes.append(predicted_class)
    confidences.append(confidence)
    
# Add the predictions and confidence scores as new columns in the DataFrame
input_text["Predicted_Class"] = predicted_classes
input_text["Confidence"] = confidences

# Save the updated DataFrame to a new Excel file
input_text.to_excel("filtered_output_predicted_chunks_with_confidence.xlsx", index=False)

In [67]:
input_text = pd.read_excel("filtered_output_predicted_chunks_with_confidence.xlsx")
input_text = input_text.drop(['Root_Texts','Root_Deps','Root_Heads'], axis = 1)
input_text.head(5)

,Sentences,Chunk_Texts,Predicted_Class,Confidence
0,Advanced Healthcare\n−\n3DP is being used in t...,Advanced Healthcare\n−\n3DP,0,0.999864
1,Advanced Healthcare\n−\n3DP is being used in t...,the medical sector,0,0.999973
2,Advanced Healthcare\n−\n3DP is being used in t...,lives,0,0.999964
3,Advanced Healthcare\n−\n3DP is being used in t...,organs,0,0.999955
4,Advanced Healthcare\n−\n3DP is being used in t...,the\nhuman body,0,0.999978


In [68]:
# Save the updated DataFrame to a new Excel file
input_text.to_excel("filtered_output_predicted_chunks_Texts_with_confidence.xlsx", index=False)

In [69]:
import pandas as pd

#Load the CSV file 
df = pd.read_excel("filtered_output_predicted_chunks_Texts_with_confidence.xlsx")

#Filter rows with Predicted_Class = 1
filtered_df = df[df["Predicted_Class"] == 1][["Sentences", "Chunk_Texts", "Predicted_Class", "Confidence"]]

filtered_df.to_excel("filtered_chunks_Texts_For_BERT.xlsx", index=False)

In [ ]:
#DONE 

In [ ]:
#Sub-tokens 

In [156]:
import evaluate
import sentencepiece as spm

# Load BLEU metric
bleu = evaluate.load("bleu")

# Load sub-tokenizer model
spm.SentencePieceTrainer.train(input='Chunk_Texts_Output_22DocMethod 2.txt', model_prefix='spm', vocab_size=200)
sp = spm.SentencePieceProcessor(model_file="spm.model")  # Load the trained model

# Read text data
with open("Chunk_Texts_Output_22DocMethod 2.txt", "r", encoding="utf-8", errors='replace') as file:
    lines = file.readlines()

# Separate predictions and references
predictions = [line.strip() for line in lines[:164]]  # Adjust based on correct count
references = [line.strip() for line in lines[164:]]   # Adjust based on correct count

# Sub-tokenize predictions and references
tokenized_predictions = [sp.encode(p, out_type=str) for p in predictions]
tokenized_references = [sp.encode(r, out_type=str) for r in references]

# Join sub-tokens back to strings (ensure each is a single string)
joined_predictions = ["".join(tokens).replace("▁", " ").strip() for tokens in tokenized_predictions]
joined_references = ["".join(tokens).replace("▁", " ").strip() for tokens in tokenized_references]

# Convert to lowercase to reduce case sensitivity issues
joined_predictions = [p.lower() for p in joined_predictions]
joined_references = [r.lower() for r in joined_references]

# Ensure alignment between predictions and references (adjust if mismatch)
if len(joined_predictions) != len(joined_references):
    min_len = min(len(joined_predictions), len(joined_references))
    joined_predictions = joined_predictions[:min_len]
    joined_references = joined_references[:min_len]

# Compute BLEU score
results = bleu.compute(predictions=joined_predictions, references=joined_references)
print("BLEU Score with Sub-tokenization:", results)


BLEU Score with Sub-tokenization: {'bleu': 0.0, 'precisions': [0.10493827160493827, 0.0, 0.0, 0.0], 'brevity_penalty': 0.5118349411833042, 'length_ratio': 0.5988909426987061, 'translation_length': 324, 'reference_length': 541}


In [157]:
joined_predictions[:5] 

['adjustment hij',
 'alum amended',
 'alum gypsum',
 'amended gypsum',
 'amended unamended']

In [158]:
joined_references[:5]

['flood risk management',
 'the isar river case study francesco pugliese',
 'ecosystem and user-friendly flood risk management plan',
 'the flood risk',
 'aft assessment framework tool drr disaster']

In [159]:
#Sub-tokens ROUGE Score 

In [160]:
import evaluate
import sentencepiece as spm

# Load ROUGE metric
rouge = evaluate.load("rouge")

# Load sub-tokenizer model
spm.SentencePieceTrainer.train(input='Chunk_Texts_Output_22DocMethod 2.txt', model_prefix='spm', vocab_size=200)
sp = spm.SentencePieceProcessor(model_file="spm.model")  # Load the trained model

# Read text data
with open("Chunk_Texts_Output_22DocMethod 2.txt", "r", encoding="utf-8", errors='replace') as file:
    lines = file.readlines()

# Separate predictions and references
predictions = [line.strip() for line in lines[:164]]  # Adjust based on correct count
references = [line.strip() for line in lines[164:]]   # Adjust based on correct count

# Sub-tokenize predictions and references
tokenized_predictions = [sp.encode(p, out_type=str) for p in predictions]
tokenized_references = [sp.encode(r, out_type=str) for r in references]

# Join sub-tokens back to strings (ensure each is a single string)
joined_predictions = ["".join(tokens).replace("▁", " ").strip() for tokens in tokenized_predictions]
joined_references = ["".join(tokens).replace("▁", " ").strip() for tokens in tokenized_references]

# Convert to lowercase to reduce case sensitivity issues
joined_predictions = [p.lower() for p in joined_predictions]
joined_references = [r.lower() for r in joined_references]

# Ensure alignment between predictions and references (adjust if mismatch)
if len(joined_predictions) != len(joined_references):
    min_len = min(len(joined_predictions), len(joined_references))
    joined_predictions = joined_predictions[:min_len]
    joined_references = joined_references[:min_len]

# Compute ROUGE score
results = rouge.compute(predictions=joined_predictions, references=joined_references)
print("ROUGE Score with Sub-tokenization:", results)


ROUGE Score with Sub-tokenization: {'rouge1': 0.08353658536585368, 'rouge2': 0.0, 'rougeL': 0.08144599303135892, 'rougeLsum': 0.08228803716608599}


In [155]:
import sentencepiece as spm

# Load sub-tokenizer model
spm.SentencePieceTrainer.train(input='Chunk_Texts_Output_22DocMethod 2.txt', model_prefix='spm', vocab_size=200)
sp = spm.SentencePieceProcessor(model_file="spm.model")  # Load the trained model

# Read text data
with open("Chunk_Texts_Output_22DocMethod 2.txt", "r", encoding="utf-8", errors='replace') as file:
    lines = file.readlines()

# Separate predictions and references
predictions = [line.strip() for line in lines[:164]]  # Adjust based on correct count
references = [line.strip() for line in lines[164:]]   # Adjust based on correct count

# Sub-tokenize predictions and references
tokenized_predictions = [sp.encode(p, out_type=str) for p in predictions]
tokenized_references = [sp.encode(r, out_type=str) for r in references]

# Print the sub-tokens for a few examples
print("Sub-tokens for Predictions:")
for i, tokens in enumerate(tokenized_predictions[:5]):  # Print first 5 tokenized predictions
    print(f"Prediction {i+1}: {tokens}")

print("\nSub-tokens for References:")
for i, tokens in enumerate(tokenized_references[:5]):  # Print first 5 tokenized references
    print(f"Reference {i+1}: {tokens}")


Sub-tokens for Predictions:
Prediction 1: ['▁flash', '▁floods', '▁flooding']
Prediction 2: ['▁floods', '▁floodplains', '▁alluvial']
Prediction 3: ['▁', 'ic', 'e', '▁', 'ja', 'm', '▁flooding']
Prediction 4: ['▁floodplains', '▁alluvial', '▁fa', 'ns']
Prediction 5: ['▁co', 'asta', 'l', '▁flooding', '▁erosion']

Sub-tokens for References:
Reference 1: ['▁Flood', 's']
Reference 2: ['▁Floodplain', 's']
Reference 3: ['▁Flood', 's']
Reference 4: ['▁"', 'Floodplains', '▁', 'C', 'hapter', '▁Over', 'v', 'i', 'e', 'w', '"']
Reference 5: ['▁floods']


In [131]:
import pandas as pd

# Load the Excel file
df = pd.read_excel("filtered_chunks_Texts_For_BERT.xlsx")

# Extract only the "Chunk_Texts" column
chunk_texts = df["Chunk_Texts"]

# Remove extra spaces and newlines, and join all values into one line
chunk_texts_cleaned = chunk_texts.str.replace('\n', ' ').str.replace('\r', '').str.replace('  ', ' ')

# Save the cleaned data to a text file
chunk_texts_cleaned.to_csv("Chunk_Texts_Output.txt", index=False, header=False, sep="\n", encoding="utf-8")


In [ ]:
#Improved 

In [169]:
import evaluate
import sentencepiece as spm
import os

# Load evaluation metrics
bleu = evaluate.load("bleu")
rouge = evaluate.load("rouge")

# Ensure the text file exists before proceeding
file_path = "Chunk_Texts_Output_22DocMethod 2.txt"
if not os.path.exists(file_path):
    raise FileNotFoundError(f"File {file_path} not found.")

# Train SentencePiece model (if not already trained)
model_prefix = "spm"
model_file = f"{model_prefix}.model"

if not os.path.exists(model_file):
    spm.SentencePieceTrainer.train(input=file_path, model_prefix=model_prefix, vocab_size=200)

# Load sub-tokenizer model
sp = spm.SentencePieceProcessor(model_file=model_file)

# Read text data
with open(file_path, "r", encoding="utf-8", errors='replace') as file:
    lines = [line.strip() for line in file.readlines() if line.strip()]

# Separate predictions and references
predictions = [line.strip() for line in lines[:164]]  # Adjust based on correct count
references = [line.strip() for line in lines[164:]]   # Adjust based on correct count

# Sub-tokenization
def subtokenize(texts):
    return [" ".join(sp.encode(p, out_type=str)).replace("▁", " ").strip().lower() for p in texts]

tokenized_predictions = subtokenize(predictions)
tokenized_references = subtokenize(references)

# Convert to lowercase to reduce case sensitivity issues
joined_predictions = [p.lower() for p in joined_predictions]
joined_references = [r.lower() for r in joined_references]

# Ensure alignment
min_len = min(len(tokenized_predictions), len(tokenized_references))
tokenized_predictions = tokenized_predictions[:min_len]
tokenized_references = tokenized_references[:min_len]

# Compute BLEU with smoothing
bleu_results = bleu.compute(predictions=tokenized_predictions, references=tokenized_references, smooth=True)

# Compute ROUGE-L
rouge_results = rouge.compute(predictions=tokenized_predictions, references=tokenized_references)

# Print results
print("\nEvaluation Results with Sub-tokenization:")
print(f"BLEU Score: {bleu_results}")
print(f"ROUGE Scores: {rouge_results}")


Evaluation Results with Sub-tokenization:
BLEU Score: {'bleu': 0.006377452429655635, 'precisions': [0.13688610240334378, 0.008827238335435058, 0.003179650238473768, 0.001941747572815534], 'brevity_penalty': 0.686211741600641, 'length_ratio': 0.7264437689969605, 'translation_length': 956, 'reference_length': 1316}
ROUGE Scores: {'rouge1': 0.11745478763182449, 'rouge2': 0.00448556209700028, 'rougeL': 0.10723539612054034, 'rougeLsum': 0.10708907510682877}


In [165]:
import evaluate
import sentencepiece as spm
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import jaccard_score
import numpy as np

# Load evaluation metrics
bleu = evaluate.load("bleu")
rouge = evaluate.load("rouge")
bertscore = evaluate.load("bertscore")

# Load sub-tokenizer model
spm.SentencePieceTrainer.train(input='Chunk_Texts_Output_22DocMethod 2.txt', model_prefix='spm', vocab_size=200)
sp = spm.SentencePieceProcessor(model_file="spm.model")  # Load the trained model

# Read text data
with open("Chunk_Texts_Output_22DocMethod 2.txt", "r", encoding="utf-8", errors='replace') as file:
    lines = file.readlines()

# Separate predictions and references
predictions = [line.strip() for line in lines[:164]]  # Adjust based on correct count
references = [line.strip() for line in lines[164:]]   # Adjust based on correct count

# Sub-tokenize predictions and references
tokenized_predictions = [sp.encode(p, out_type=str) for p in predictions]
tokenized_references = [sp.encode(r, out_type=str) for r in references]

# Join sub-tokens back to strings (ensure each is a single string)
joined_predictions = ["".join(tokens).replace("▁", " ").strip() for tokens in tokenized_predictions]
joined_references = ["".join(tokens).replace("▁", " ").strip() for tokens in tokenized_references]

# Convert to lowercase to reduce case sensitivity issues
joined_predictions = [p.lower() for p in joined_predictions]
joined_references = [r.lower() for r in joined_references]

# Ensure alignment between predictions and references (adjust if mismatch)
if len(joined_predictions) != len(joined_references):
    min_len = min(len(joined_predictions), len(joined_references))
    joined_predictions = joined_predictions[:min_len]
    joined_references = joined_references[:min_len]

# Compute BLEU score
bleu_results = bleu.compute(predictions=joined_predictions, references=joined_references)
print("BLEU Score with Sub-tokenization:", bleu_results)

# Compute ROUGE score
rouge_results = rouge.compute(predictions=joined_predictions, references=joined_references)
print("ROUGE Score with Sub-tokenization:", rouge_results)

# Compute BERTScore
bertscore_results = bertscore.compute(predictions=joined_predictions, references=joined_references, lang="en")
average_bertscore = np.mean(bertscore_results['f1'])
print("BERTScore (F1):", average_bertscore)

# Compute Cosine Similarity using Sentence Embeddings
model = SentenceTransformer("all-MiniLM-L6-v2")  # Load a compact embedding model
pred_embeddings = model.encode(joined_predictions)
ref_embeddings = model.encode(joined_references)

similarity_scores = cosine_similarity(pred_embeddings, ref_embeddings)
average_cosine_similarity = similarity_scores.diagonal().mean()
print("Average Cosine Similarity:", average_cosine_similarity)

# Compute Jaccard Similarity
vectorizer = CountVectorizer(binary=True).fit(joined_predictions + joined_references)
pred_vectors = vectorizer.transform(joined_predictions)
ref_vectors = vectorizer.transform(joined_references)

jaccard_scores = [
    jaccard_score(pred_vectors[i].toarray()[0], ref_vectors[i].toarray()[0])
    for i in range(len(joined_predictions))
]
average_jaccard_similarity = sum(jaccard_scores) / len(jaccard_scores)
print("Average Jaccard Similarity:", average_jaccard_similarity)


ImportError: cannot import name 'cached_download' from 'huggingface_hub' (C:\Users\salsubhi1\Anaconda3\envs\geo_env\lib\site-packages\huggingface_hub\__init__.py)

In [184]:
# Read the file and extract sets
with open("SetAandSetB_6DocMethod1.txt", "r", encoding="utf-8") as file:
    data = file.read()

# Extract sets using string manipulation
set_a_str = data.split("Set A:")[1].split("Set B:")[0].strip()
set_b_str = data.split("Set B:")[1].strip()

# Convert string representations to actual sets (case insensitive)
set_a = set(word.strip().lower() for word in set_a_str.strip("{}\n").replace("\n", "").split(", "))
set_b = set(word.strip().lower() for word in set_b_str.strip("{}\n").replace("\n", "").split(", "))

# Calculate intersection
intersection = set_a.intersection(set_b)

# Calculate percentage of intersection
percentage = (len(intersection) / len(set_a)) * 100 if set_a else 0

# Print the results
print("Set A:", set_a)
print(" ")
print("Set B:", set_b)
print(" ")
print("Intersection:", intersection)
print(" ")
print(f"Percentage of intersection: {percentage:.2f}%")

Set A: {'events', 'pattern', 'summer', 'perception', 'long', 'winter', 'assessment', 'risk', 'floodplains', 'types', 'rain', 'soil', 'management', 'moisture', 'flash', 'snowmelt', 'vulnerability', 'flood'}
 
Set B: {'build-up', 'preparedness', 'related', 'research', 'risks', 'forecasting', 'approaches', 'additional', 'slow-moving', 'overview', 'community', 'tide', 'shallow', 'resilience', 'small', 'catastrophic', 'static', 'rain', 'estimates', 'orographic', 'serious', 'zones', 'maximum', 'exposure', 'progression', 'sample', 'wave', 'elevation', 'surge', 'extensive', 'prevention', 'capacity', 'observed', 'affected', 'factors', 'frequency', 'mitigation', 'techniques', 'pre-storm', 'future', 'interventions', 'type', 'dynamics', 'increased', 'model', 'associated', 'distributions', 'dead', 'defences', 'data', 'timescale', 'problems', 'erosion', 'inundation', 'snowmelt-induced', 'bangladesh', 'hydro-meteorological', 'tropical', 'awareness', 'constructivist-oriented', 'duration', 'debris', 's